# Import Libraries

In [ ]:
import sys
import os
import re
import hashlib
import sqlite3

# SQL Database

In [ ]:
# Create database 
def create_db(db_path): # Called from Main()
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    cur.execute('''CREATE TABLE login
                   (username text, password text)''') # Create table 'login' with columns username and password
    con.commit()
    con.close()
    
def check_un(un): # Called from raw_input()
    exist = False # Variable default to false indicating account doesnt exist
    con = sqlite3.connect('login_db.db')
    cur = con.cursor()
    cur.execute('''SELECT * FROM login where username=:un''', {"un": un}) # Select row based on username input
    un_db = cur.fetchall()
    if len(un_db) != 0: # If table not empty, username exist
        exist = True
    con.commit()
    con.close()
    return exist
    
def create_login(un, hash_pw): # Called from account()
    con = sqlite3.connect('login_db.db')
    cur = con.cursor()
    cur.execute('''INSERT INTO login VALUES (?, ?)''', (un, hash_pw))# Insert new username and hashed password row to login table
    con.commit()
    con.close()
    print("Account created")

        
def search_login(un, hash_pw): # Called from account()
    login = False # Variable to indicated login status
    con = sqlite3.connect('login_db.db')
    cur = con.cursor()
    cur.execute('''SELECT * FROM login where username=:un''', {"un": un}) # Select row based on username input
    login = cur.fetchall()
    for _ in login:
        if _[1] == hash_pw: # Match user password input with database password
            login = True
    con.commit()
    con.close()    
    if login:
        print("Login successful")
    else:
        print("Check username or password")
    return login # TODO: return login token if required

def delete_account(un, hash_pw):
    status = search_login(un, hash_pw)
    
    if status:
        confirm_del = str(input("Please confirm by account deletion by entering password: "))
        confirm_hash = hash_legal(confirm_del)
        confirm = False
        
        con = sqlite3.connect('login_db.db')
        cur = con.cursor()
        cur.execute('''SELECT * FROM login where username=:un''', {"un": un}) # Select row based on username input
        login = cur.fetchall()
        for _ in login:
            if _[1] == confirm_hash: # Match user password input with database password
                confirm = True
        
        if confirm:
            cur.execute('''DELETE FROM login where username=:un''', {"un": un})

        con.commit()
        con.close()
        
        return("Account deleted")

    else:
        return "Password does not match, account not removed."
                


# User input

In [ ]:
# Ask for user input
def raw_input(option): # Called from account()
    while True:
        raw_un = str(input("Please enter username: ")) # User input
        if check_un(raw_un): # If account exist in database
            if option == 'create':
                print("Please enter a different username.")
            else:
                break
        else: # If account does notexist in database
            if option == 'login':
                print("Account doesnt exist.")
            else:
                break
    while True:
        raw_pw = str(input("Please enter password: ")) # User input
        if check_illegal(raw_pw) == False: # Check if password matches requirements
            break   
    hash_pw = hash_legal(raw_pw) # Hash legal password
    return raw_un, hash_pw

# Sanitisation
def check_illegal(raw_pw): # Called from and returned to raw_input()
    # Check for empty string
    if not raw_pw:
        print("Password field cannot be empty")
        return True
    # Check password length
    elif len(raw_pw) <= 7:
        print("Password length must be more than 8 characters")
        return True
    # Checks for at least 1 number, 1 alpha(both upper and lower)
    elif re.search('^(?=.*?\d)(?=.*?[a-z])(?=.*?[A-Z])(?=.*?[^A-Za-z\s0-9])', raw_pw) is None:
        print("Password must contain at least 1 uppercase and lowercase letter, number and special character.")
        return True
    # Input is legal
    else:    
        return False

# Hashing passwords using SHA256
def hash_legal(legal_input): # Called from raw_input()
    return hashlib.sha256(legal_input.encode('utf-8')).hexdigest()

# Create or Login to Account

In [ ]:
def account(option): # Called from Main()
    un, hash_pw = raw_input(option)
    if option == 'login':
        search_login(un, hash_pw)
    elif option == 'create':
        create_login(un, hash_pw)
    elif option == 'delete':
        print(delete_account(un, hash_pw))

# Main

In [ ]:
def main():
    # Check if database exist, otherwise create database.
    db_path = 'login_db.db'
    if not os.path.exists(db_path):
        create_db(db_path)
    
    # Main Function
    while True:
        option = (input("Please select from 'create', 'login', 'delete' or 'exit': ")).lower().strip()
        if option == "exit":
            print("Exiting program")
            break
        elif option == 'create': 
            account('create')
            break
        elif option == 'login':
            account('login')
            break
        elif option == 'delete':
            account('delete')
            break
        else:
            print("Please select correct option")
            
if __name__ == "__main__":
    main()
    print("Program ran to completion")
    